In [3]:
import os
from tifffile import imread
import numpy as np
from scipy.ndimage.interpolation import rotate as rotate_image
from tifffile import imsave
from numba import jit
from tqdm.notebook import tqdm as tqdm

root_path = '/archive/MIL/friedman/Yeast/mic60GFP-MitoDsRED-MDM34JF646/200610/Cell1_decon'
data_path = os.path.join(root_path, '1_CH00_000000.tif')
test = imread(data_path)

test = np.array(test);
shape = test.shape
print(shape)
# t, z, c, x, y

(50, 51, 3, 256, 1600)


In [2]:
def deskew(inArray, angle, dz, xypixelsize):
    (z_len, y_len, x_len) = inArray.shape
    Trans = np.cos(angle * np.pi / 180) * dz / xypixelsize
    widenBy = np.uint16(np.ceil(z_len * np.cos(angle * np.pi / 180) * dz / xypixelsize))

    inArrayWiden = np.zeros((z_len, y_len, x_len + widenBy))
    inArrayWiden[:z_len, :y_len, :x_len] = inArray
    output = np.zeros((z_len, y_len, x_len + widenBy))
    xF, yF = np.meshgrid(np.arange(x_len + widenBy), np.arange(y_len))

    for k in range(z_len):
        inSlice = inArrayWiden[k, :, :]
        inSliceFFT = np.fft.fftshift(np.fft.fft2(inSlice))
        inSliceFFTTrans = inSliceFFT * np.exp(-1j * 2 * np.pi * xF * Trans * k / (x_len + widenBy))
        output_temp = np.abs(np.fft.ifft2(np.fft.ifftshift(inSliceFFTTrans)))
        output[k, :, :] = output_temp

    output[output < 0] = 0
    return np.uint16(output)  # return uint16 data to save as tiff


In [5]:
counter = 1
for k in tqdm(range(5)):
    root_path = '/archive/MIL/friedman/Yeast/mic60GFP-MitoDsRED-MDM34JF646/200610/Cell' + str(counter) + '_decon'
    print(root_path)
    counter = counter + 1
    
    export_path = root_path + "_sheared"
    if os.path.isdir(export_path):
        print("Folder Already Exists")
    else:
        os.mkdir(export_path)

    data_path = os.path.join(root_path, '1_CH00_000000.tif')
    test = imread(data_path)

    test = np.array(test);
    shape = test.shape
    print(shape)

# t, z, c, x, y
    for i in tqdm(range(shape[2])):
        for j in tqdm(range(shape[0])):
            inArray = test[j,:,i,:,:]
            inArray = inArray - np.amin(inArray)
            inArray = rotate_image(inArray, 90, axes=(1,2))
            test_deskewed = deskew(inArray, 26, 400, 115)
            export_name = os.path.join(export_path, '1_CH0' + str(i) + '_' + str(("{:06d}".format(j))) + '.tif')
            imsave(export_name, test_deskewed)

/archive/MIL/friedman/Yeast/mic60GFP-MitoDsRED-MDM34JF646/200610/Cell1_decon
(50, 51, 3, 256, 1600)




/archive/MIL/friedman/Yeast/mic60GFP-MitoDsRED-MDM34JF646/200610/Cell2_decon
(50, 51, 3, 256, 1600)




/archive/MIL/friedman/Yeast/mic60GFP-MitoDsRED-MDM34JF646/200610/Cell3_decon
(50, 51, 3, 256, 1600)




/archive/MIL/friedman/Yeast/mic60GFP-MitoDsRED-MDM34JF646/200610/Cell4_decon
(50, 51, 3, 256, 1600)




/archive/MIL/friedman/Yeast/mic60GFP-MitoDsRED-MDM34JF646/200610/Cell5_decon
(50, 51, 3, 256, 1600)


In [10]:
root_path = '/archive/MIL/friedman/Yeast/mic60GFP-MitoDsRED/200603/Cell1_decon'

export_path = root_path + "_sheared"
if os.path.isdir(export_path):
    print("Folder Already Exists")
else:
    os.mkdir(export_path)

data_path = os.path.join(root_path, '1_CH00_000000.tif')
test = imread(data_path)

test = np.array(test);
shape = test.shape
print(shape)

(75, 51, 2, 256, 2048)


In [11]:
for channel in tqdm(range(shape[2])):
    for time in tqdm(range(shape[0])):
        inArray = test[time,:,channel,:,:]
        inArray = inArray - np.amin(inArray)
        inArray = rotate_image(inArray, 90, axes=(1,2))
        test_deskewed = deskew(inArray, 26, 400, 115)
        export_name = os.path.join(export_path, '1_CH0' + str(channel) + '_' + str(("{:06d}".format(time))) + '.tif')
        imsave(export_name, test_deskewed)

In [14]:
root_path = '/archive/MIL/friedman/Yeast/Mic60GFP/200603/Cell3_decon'

export_path = root_path + "_sheared"
if os.path.isdir(export_path):
    print("Folder Already Exists")
else:
    os.mkdir(export_path)

data_path = os.path.join(root_path, '1_CH00_000000.tif')
test = imread(data_path)

test = np.array(test);
shape = test.shape
print(shape)

(120, 51, 256, 2048)


In [15]:
for time in tqdm(range(shape[0])):
    inArray = test[time,:,:,:]
    inArray = inArray - np.amin(inArray)
    inArray = rotate_image(inArray, 90, axes=(1,2))
    test_deskewed = deskew(inArray, 26, 400, 115)
    export_name = os.path.join(export_path, '1_CH0' + str(channel) + '_' + str(("{:06d}".format(time))) + '.tif')
    imsave(export_name, test_deskewed)